In [1]:
#DQN Agent
import sys
pwd = "P:/dsv/DAMII/Practical Project/rl_stock_agents"
sys.path.append(pwd)

import cleandata
import numpy as np
from stockenv import ContinuousOHLCVEnv
import matplotlib.pyplot as plt
from agentperform import agent_stock_performance
from agents import DQNAgent
import logging



stock_list =["amzn_daily.csv"]#, "appl_daily.csv","tsla_daily.csv","f_daily.csv","pfe_daily.csv",
             #"coke_daily.csv","brk_daily.csv","nee_daily.csv","jnj_daily.csv","pg_daily.csv"]

stock_name_list = ['AMZN']#,'APPL','TSLA','F','PFE','COKE','BRK','NEE','JNJ','PG']

import_path = "P:/dsv/DAMII/Practical Project/rl_stock_agents/input_data"


# Instantiate the environment

results =[]

for file_name, stock_name in zip(stock_list,stock_name_list):
    # Import File
    
    df_ohlcv = cleandata.NASDAQ_csv_input(file_name,import_path)


    env = ContinuousOHLCVEnv(df_ohlcv[["open","high","low",'close',"volume"]].iloc[:2000].to_numpy())


    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    available_actions = env.available_actions

    # Create a DQN agent
    agent = DQNAgent(state_size, action_size,available_actions)

    batch_size = 32
    EPISODES = 50  # Number of episodes for training

    for e in range(EPISODES):
        logging.info(e)
        env.reset()
        state = env.get_observation()
        state = np.reshape(state, [1, state_size])
        print(available_actions)
        for time in range(500):  # Adjust the number of time steps per episode
            action, action_type = agent.act(state)
            next_state, reward, new_available_actions, done = env.step(action)
            logging.info(action_type)
            logging.info(env.step_info[-1])
            agent.available_actions = new_available_actions
            reward = reward if not done else -10  # Modify the reward as needed
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                print(f"episode: {e}/{EPISODES}, score: {time}, epsilon: {agent.epsilon:.2}")
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)



(0, 1)
1/1 [==============================] - 0s 126ms/step


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()